In [ ]:
#import required libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.svm import OneClassSVM
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest

In [ ]:
# URL to import data set.
url = ''

In [ ]:
df = pd.read_csv(url)

In [ ]:
df.head()

In [ ]:
df.info()

# Data Cleaning

In [ ]:
#identifying duplicate values
df.duplicated().any()

In [ ]:
#identifying missing values
df.isnull().sum()

In [ ]:
#visualise correlation heatmap
corr_matrix = df.corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.show()

# Descriptive statistics

In [ ]:
df.describe()

# Exploratory data analysis

In [ ]:
plt.figure(figsize=(18,10))
plt.subplot(2, 3, 1)
plt.hist(df['Engine rpm'], bins=20, color='skyblue', edgecolor='black')
plt.title('Histogram of Engine rpm')
plt.xlabel('Engine rpm')
plt.ylabel('Frequency')

plt.subplot(2, 3, 2)
plt.hist(df['Lub oil pressure'], bins=20, color='salmon', edgecolor='black')
plt.title('Histogram of Lub oil pressure')
plt.xlabel('Lub oil pressure')
plt.ylabel('Frequency')

plt.subplot(2, 3, 3)
plt.hist(df['Fuel pressure'], bins=20, color='lightgreen', edgecolor='black')
plt.title('Histogram of Fuel pressure')
plt.xlabel('Fuel pressure')
plt.ylabel('Frequency')

plt.subplot(2, 3, 4)
plt.hist(df['Coolant pressure'], bins=20, color='orange', edgecolor='black')
plt.title('Histogram of Coolant pressure')
plt.xlabel('Coolant pressure')
plt.ylabel('Frequency')

plt.subplot(2, 3, 5)
plt.hist(df['lub oil temp'], bins=20, color='purple', edgecolor='black')
plt.title('Histogram of Lub oil temp')
plt.xlabel('lub oil temp')
plt.ylabel('Frequency')

plt.subplot(2, 3, 6)
plt.hist(df['Coolant temp'], bins=20, color='lightyellow', edgecolor='black')
plt.title('Histogram of Coolant temp')
plt.xlabel('Coolant temp')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(18, 10))
plt.subplot(2, 3, 1)
plt.boxplot(df['Engine rpm'])
plt.title('Box Plot of Engine rpm')

plt.subplot(2, 3, 2)
plt.boxplot(df['Lub oil pressure'])
plt.title('Box Plot of Lub oil pressure')

plt.subplot(2, 3, 3)
plt.boxplot(df['Fuel pressure'])
plt.title('Box Plot of Fuel pressure')

plt.subplot(2, 3, 4)
plt.boxplot(df['Coolant pressure'])
plt.title('Box Plot of Coolant pressure')

plt.subplot(2, 3, 5)
plt.boxplot(df['lub oil temp'])
plt.title('Box Plot of lub oil temp')

plt.subplot(2, 3, 6)
plt.boxplot(df['Coolant temp'])
plt.title('Box Plot of Coolant temp')


plt.tight_layout()
plt.show()

# Anomaly Detection

## Using interquartile range (IQR) method to identify outliers for each feature.

In [ ]:
outlier_df = df.copy()

In [ ]:
for col in outlier_df.columns:

    Q1 = outlier_df[col].quantile(0.25)
    Q3 = outlier_df[col].quantile(0.75)
    IQR = Q3 - Q1

    outlier_df[f'Outlier_{col}'] = ((outlier_df[col] < (Q1 - 1.5 * IQR)) | (outlier_df[col] > (Q3 + 1.5 * IQR))).astype(int)

outlier_df


In [ ]:
# filtering out samples
outliers_count = outlier_df.filter(like='Outlier').sum(axis=1)
outlier_df['anomaly'] = (outliers_count >= 2).astype(int)
outlier_df

In [ ]:
# outlier percentage
outlier_percentage = (outlier_df['anomaly'].sum()/len(outlier_df))*100
outlier_percentage

In [ ]:
# checking samples who have all features under outlier category
outlier_df[outliers_count == len(df.columns)]

## Anomaly detection using OneClassSVM

Feature Scaling

In [ ]:
# Using MinMaxScaler
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df)
scaled_df = pd.DataFrame(scaled_data, columns=df.columns)

scaled_df.head()

Now lets apply one-class SVM to the scaled dataframe.

In [ ]:
model = OneClassSVM(kernel='rbf', gamma=0.2, nu=0.03)
model.fit(scaled_df)

In [ ]:
anomaly = model.predict(scaled_df)
anomaly

In [ ]:
svm_df = df.copy()
svm_df['anomaly']= anomaly
svm_df



> Predicting anomalies where '-1' indicates an anomaly and '1' indicates normal behavior.



In [ ]:
# Display only anomalies in the DataFrame.
only_anomalies = svm_df[svm_df.anomaly == -1]
print(only_anomalies.shape)
only_anomalies

In [ ]:
# outlier percentage
outlier_percentage = ((svm_df['anomaly'] == -1).sum()/len(svm_df))*100
outlier_percentage

In [ ]:
scaled_df[features].head()

In [ ]:
# Using PCA for dimensionality reduction
pca = PCA(n_components = 2)
components = pca.fit_transform(scaled_df[features])
components[:5]

In [ ]:
# creating dataframe for PCA model data
pca_df = pd.DataFrame(data = components,columns=['pca_1','pca_2'])
pca_df.head()

In [ ]:
# for 2D visualisation
model = OneClassSVM(kernel='rbf', gamma=0.2, nu=0.03)
model.fit(pca_df)

plt.figure(figsize=(12, 8))
plt.title('Anomaly Detection')

xx, yy = np.meshgrid(np.linspace(-0.8, 0.8, 500), np.linspace(-0.8, 0.8, 500))
Z = model.decision_function(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

y_pred = model.predict(pca_df)

plt.contourf(xx, yy, Z, levels=np.linspace(Z.min(), 0, 7), cmap=plt.cm.PuBu, alpha=0.8)
plt.contour(xx, yy, Z, levels=[0], linewidths=2, colors='darkred')  # Decision boundary
plt.contourf(xx, yy, Z, levels=[0, Z.max()], colors='palevioletred', alpha=0.5)

sns.scatterplot(x=pca_df['pca_1'], y=pca_df['pca_2'], hue=np.where(y_pred == 1, 'Normal', 'Anomaly'),
                    style=np.where(y_pred == -1, 'Anomaly', 'Normal'), markers={'Anomaly': 'X', 'Normal': 'o'},
                    palette={'Normal': 'deepskyblue', 'Anomaly': 'red'}, alpha=0.6, edgecolor='k')


plt.axis('tight')
plt.xlim((-0.8, 0.8))
plt.ylim((-0.8, 0.8))
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.legend()
plt.show()

## Detecting anomalies using Isolation Forest algorithm

In [ ]:
iso_forest = IsolationForest(contamination=0.03, random_state=42)
iso_forest.fit(scaled_df)

In [ ]:
# predicting anomalies
outlier = iso_forest.predict(scaled_df)
outlier

In [ ]:
iso_df = df.copy()
iso_df['anomaly'] = outlier
iso_df

In [ ]:
((iso_df['anomaly'] == -1).sum()/len(svm_df))*100

For visualising the anomalies generated in isolation forest model in 2D, I used PCA model to get 2D data.

In [ ]:
pcaiso_forest = IsolationForest(contamination=0.03, random_state=42)
pcaiso_forest.fit(pca_df)

In [ ]:
outlier = pcaiso_forest.predict(pca_df)
outlier

In [ ]:
pcaiso_df = df.copy()
pcaiso_df['anomaly'] = outlier
pcaiso_df

In [ ]:
# Display only anomalies in the DataFrame.
only_anomalies = pcaiso_df[pcaiso_df.anomaly == -1]
print(only_anomalies.shape)
only_anomalies

In [ ]:
updated_df = pd.concat([pca_df,pcaiso_df['anomaly']],axis=1)
updated_df

In [ ]:
# visualising the output in 2D
plt.figure(figsize=(12, 8))
plt.scatter(updated_df['pca_1'], updated_df['pca_2'], c= updated_df['anomaly'], cmap='coolwarm')
plt.title('Isolation Forest Anomaly Detection (2D PCA)')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.colorbar(label='Outlier (1: Normal, -1: Anomaly)')
plt.show()

# Reflect

> The objective was to develop a robust anomaly detection system to protect the shipping fleet by evaluating engine functionality. The dataset had six important features to evaluate the engine’s status as good or bad. I decided to follow a structured approach of exploratory data analysis, data preprocessing and anomaly detection using statistical methods like IQR and also using ML models like one-class SVM and Isolation Forest. The 2D visualisation of anomalies by reducing the six features to two using the PCA model will give me a better insight about the distribution of anomalies and normal data behaviour. The visualisation and outputs from the model can then be used to interpret which ships are functioning badly and features that need to be monitored and fixed.


### Reference:
Devabrat, M., 2022. Predictive Maintenance on Ship's Main Engine using AI. Available at: https://dx.doi.org/10.21227/g3za-v415. [Accessed 5 March 2024]